In [1]:
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from statistics import median

### **Utils**

In [2]:
def remove_outliers(df: pd.DataFrame, col: str) -> pd.DataFrame:

    mean_bound = df[col].mean()
    outliers_mask = (df[col] < mean_bound)

    df = df[~outliers_mask]

    return df

In [3]:
def plot_death_by_causes(df, title, x='causes', y='quantity'):
    fig = px.bar(df, x=x, y=y)

    fig.update_layout(
        xaxis=dict(categoryorder='total descending'),
        title=title
    )
    fig.show()

In [4]:
def get_dataframe(df_name: str) -> pd.DataFrame():
    return pd.read_csv(f'./data/valid/{df_name}.csv')

### **Quais são os municípios com as maiores taxas de óbitos por causas evitáveis em crianças de 0 a 5 anos?**

In [5]:
df_born = get_dataframe('df_born')
df_born['quantity'] = df_born.drop('municipio', axis=1).sum(axis=1)
df_born.head()
# df_childs = df_childs.sum(axis=1).reset_index(name='city').rename(columns={'city': 'quantity', 'index': 'city'})

,municipio,0 a 6 dias,7 a 27 dias,28 a 364 dias,1 a 4 anos,quantity
0,abreu e lima,6.0,3.0,5.0,5.0,19.0
1,afogados da ingazeira,3.0,1.0,NaN,1.0,5.0
2,afranio,1.0,2.0,NaN,NaN,3.0
3,agrestina,1.0,2.0,3.0,NaN,6.0
4,agua preta,4.0,1.0,NaN,2.0,7.0


In [6]:
df_born.shape

(175, 6)

In [7]:
df_born = remove_outliers(df_born, 'quantity')
df_born.shape

(37, 6)

In [8]:
plot_death_by_causes(df_born, "Mortes por município (crianças de 0-5 anos)", x='municipio')

In [9]:
df_born_alive = get_dataframe('df_born_alive')
df_born_alive.shape

(187, 2)

In [10]:
df_born_merged = pd.merge(df_born, df_born_alive, on='municipio')

In [11]:
df_born_merged['letalidade infantil'] = (df_born_merged['0 a 6 dias'] / (df_born_merged['0 a 6 dias'] + df_born_merged['nascido vivo'])) * 100
df_born_merged['taxa de mortes evitaveis'] = (df_born_merged['0 a 6 dias'] / df_born_merged['nascido vivo']) * 100
df_born_merged.head()

,municipio,0 a 6 dias,7 a 27 dias,28 a 364 dias,1 a 4 anos,quantity,nascido vivo,letalidade infantil,taxa de mortes evitaveis
0,abreu e lima,6.0,3.0,5.0,5.0,19.0,1275,0.468384,0.470588
1,aguas belas,5.0,2.0,4.0,3.0,14.0,702,0.707214,0.712251
2,araripina,14.0,3.0,6.0,4.0,27.0,1616,0.858896,0.866337
3,arcoverde,9.0,2.0,10.0,3.0,24.0,1135,0.786713,0.792952
4,belo jardim,4.0,2.0,4.0,4.0,14.0,1119,0.356189,0.357462


In [12]:
# Normalizing rates
df_born_merged['letalidade infantil'] = df_born_merged['letalidade infantil'] / df_born_merged['letalidade infantil'].max()
df_born_merged['taxa de mortes evitaveis'] = df_born_merged['taxa de mortes evitaveis'] / df_born_merged['taxa de mortes evitaveis'].max()
df_born_merged.head()

,municipio,0 a 6 dias,7 a 27 dias,28 a 364 dias,1 a 4 anos,quantity,nascido vivo,letalidade infantil,taxa de mortes evitaveis
0,abreu e lima,6.0,3.0,5.0,5.0,19.0,1275,0.303781,0.300504
1,aguas belas,5.0,2.0,4.0,3.0,14.0,702,0.458679,0.454823
2,araripina,14.0,3.0,6.0,4.0,27.0,1616,0.557055,0.553218
3,arcoverde,9.0,2.0,10.0,3.0,24.0,1135,0.510240,0.506356
4,belo jardim,4.0,2.0,4.0,4.0,14.0,1119,0.231014,0.228265


In [13]:
# import folium
# import geopandas as gpd

In [14]:
# mapa = folium.Map(location=[-8.4319084, -37.60809747],
#                     tiles="OpenStreetMap", zoom_start=7.33)
# mapageografico = gpd.read_file('./data/raw/geojs-26-mun.json')

# geoJSON_muni = list(mapageografico.name.values)
# state_data = pd.read_csv(filename)

In [15]:
# babies_map = get_pe_map(df_born)
# babies_map

### **Existe uma correlação entre o número de consultas pré-natal realizadas e o peso ao nascer dos bebês em determinado grupo de municípios?**

In [16]:
df_babies_weight = pd.read_csv('./data/babies_weight.csv')

In [17]:
df_babies_weight['peso ao nascer'] = df_babies_weight['peso ao nascer'].str.split('-').map(lambda x: median([int(x[0]), int(x[1])]))

In [18]:
df_babies_weight

,peso ao nascer,nenhuma,1-3 vezes,4-6 vezes,7 e +,não informado,ignorado
0,300.0,6,33,48,87,0,4
1,749.5,23,185,305,95,0,21
2,1249.5,43,162,494,295,1,23
3,1999.5,211,813,2797,5338,4,109
4,2749.5,407,1475,5920,20305,11,228
5,3499.5,804,3349,14188,64547,48,536
6,4499.5,75,282,1113,6053,1,30


In [19]:
# Select the columns representing the values of medical consultations
consultation_columns = ['peso ao nascer', 'nenhuma', '1-3 vezes', '4-6 vezes', '7 e +', 'não informado', 'ignorado']

# Create a subset DataFrame with only the birth weight and medical consultation columns
subset_df = df_babies_weight[consultation_columns]

row_index = 'peso ao nascer'

# Calculate the correlation matrix
correlation = subset_df.corr().loc[row_index]

# Create a bar plot of the correlation values using Plotly
fig = go.Figure(data=[go.Bar(x=correlation.index, y=correlation.values)])

# Customize the layout of the plot
fig.update_layout(
    title=f'Correlation Plot of "{row_index}" with Medical Consultations',
    xaxis=dict(title='Medical Consultation Categories', tickangle=45),
    yaxis=dict(title='Correlation')
)

# Show the plot
fig.show()

### **Quais são as principais causas de óbitos evitáveis em diferentes faixas etárias nos municípios com maiores taxas de mortalidade?**

##### Crianças de 0-5 anos

In [20]:
df_causes_born = pd.read_csv('./data/born_deaths_causes.csv', index_col=0)
df_causes_born = df_causes_born.sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})

In [21]:
df_causes_born.head()

,causes,quantity
0,..reduziveis atencao a mulher na gestacao,470.0
1,..sifilis congenita,11.0
2,..feto e recem-nasc afet compl placenta membranas,31.0
3,..feto e recem-nascido afet afeccoes maternas,143.0
4,..feto e recem-nasc afet por compl matern gravid,101.0


In [22]:
df_causes_born = remove_outliers(df_causes_born, 'quantity')

In [23]:
plot_death_by_causes(df_causes_born, "Mortes por causas evitáveis (crianças de 0-5 anos)")

##### Diferentes faixas

In [24]:
df_causes_adulthood = pd.read_csv('./data/adulthood_deaths_causes.csv', index_col=0)

In [25]:
child = ['5 a 9 anos', '10 a 14 anos']
young = ['15 a 19 anos', '20 a 29 anos']
adult = ['30 a 39 anos', '40 a 49 anos', '50 a 59 anos']
elderly = ['60 a 69 anos', '70 a 74 anos']

In [26]:
df_child = df_causes_adulthood[child].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})
df_young = df_causes_adulthood[young].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})
df_adult = df_causes_adulthood[adult].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})
df_elderly = df_causes_adulthood[elderly].sum(axis=1).reset_index(name='quantity').rename(columns={'index': 'causes'})

In [27]:
df_child = remove_outliers(df_child, 'quantity')
df_young = remove_outliers(df_young, 'quantity')
df_adult = remove_outliers(df_adult, 'quantity')
df_elderly = remove_outliers(df_elderly, 'quantity')

In [28]:
plot_death_by_causes(df_child, "Mortes por causas evitáveis (5-14 anos)")

In [29]:
plot_death_by_causes(df_young, "Mortes por causas evitáveis (15-29 anos)")

In [30]:
plot_death_by_causes(df_adult, "Mortes por causas evitáveis (30-59 anos)")

In [31]:
plot_death_by_causes(df_elderly, "Mortes por causas evitáveis (60-74 anos)")

### **Quais são os municípios com os melhores indicadores de saúde básica, considerando a taxa de óbitos evitáveis e a disponibilidade de serviços básicos de saúde?**

In [32]:
df_health_centers = get_dataframe('df_health_centers')
df_health_centers.head()

,municipio,posto de saude,centro de saude/unidade basica,unidade mista
0,abreu e lima,4.0,26.0,NaN
1,afogados da ingazeira,NaN,16.0,NaN
2,afranio,1.0,12.0,NaN
3,agrestina,16.0,2.0,NaN
4,agua preta,NaN,15.0,1.0


In [33]:
numeric_col = df_health_centers.columns[1:]

df_health_centers['total de centros'] = df_health_centers[numeric_col].sum(axis=1)
df_health_centers['taxa de saude disponivel'] = (df_health_centers['total de centros'] - df_health_centers['total de centros'].mean()) / df_health_centers['total de centros'].std()

df_health_centers.head()

,municipio,posto de saude,centro de saude/unidade basica,unidade mista,total de centros,taxa de saude disponivel
0,abreu e lima,4.0,26.0,NaN,30.0,0.853577
1,afogados da ingazeira,NaN,16.0,NaN,16.0,0.032037
2,afranio,1.0,12.0,NaN,13.0,-0.144007
3,agrestina,16.0,2.0,NaN,18.0,0.149400
4,agua preta,NaN,15.0,1.0,16.0,0.032037


In [34]:
useful_cols = ['municipio', 'letalidade infantil', 'taxa de mortes evitaveis', 'total de centros', 'taxa de saude disponivel']

df_merge = pd.merge(df_born_merged, df_health_centers, on='municipio')[useful_cols]
df_merge.head()

,municipio,letalidade infantil,taxa de mortes evitaveis,total de centros,taxa de saude disponivel
0,abreu e lima,0.303781,0.300504,30.0,0.853577
1,aguas belas,0.458679,0.454823,18.0,0.149400
2,araripina,0.557055,0.553218,26.0,0.618851
3,arcoverde,0.510240,0.506356,33.0,1.029621
4,belo jardim,0.231014,0.228265,25.0,0.560170


In [35]:
df_merge['indicador de saude infantil'] = df_merge['taxa de saude disponivel'] + df_merge['taxa de mortes evitaveis']

In [36]:
df_health_child = df_merge[['municipio', 'indicador de saude infantil']]
df_health_child.head()

,municipio,indicador de saude infantil
0,abreu e lima,1.154081
1,aguas belas,0.604223
2,araripina,1.172069
3,arcoverde,1.535977
4,belo jardim,0.788435


In [37]:
fig = go.Figure(data=[go.Bar(x=df_health_child['municipio'], y=df_health_child['indicador de saude infantil'])])

# Customize the layout of the plot
fig.update_layout(
    title=f'Indicador de saúde básica considerando UBS e taxa de mortes evitáveis (0-5 anos)',
    xaxis=dict(title='Municipio', tickangle=45, categoryorder='total descending'),
    yaxis=dict(title='Taxa')
)

# Show the plot
fig.show()

### **Quais são as microrregiões com as menores taxas de óbitos por causas evitáveis na faixa etária de 5 a 70 anos?**

In [38]:
df_merged = get_dataframe('df_merged')
df_merged.head()

,municipio,populacao,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 29 anos,30 a 39 anos,40 a 49 anos,50 a 59 anos,60 a 69 anos,70 a 74 anos,microrregiao,posto de saude,centro de saude/unidade basica,unidade mista,idh
0,abreu e lima,100699,1.0,1.0,8.0,33.0,41.0,68.0,156.0,176.0,114.0,recife,4.0,26.0,NaN,NaN
1,afogados da ingazeira,37545,NaN,1.0,4.0,8.0,12.0,26.0,50.0,43.0,42.0,pajeu,NaN,16.0,NaN,NaN
2,afranio,19978,1.0,NaN,3.0,5.0,8.0,10.0,9.0,17.0,13.0,petrolina,1.0,12.0,NaN,0.588
3,agrestina,25240,NaN,NaN,5.0,8.0,20.0,32.0,15.0,36.0,25.0,brejo pernambucano,16.0,2.0,NaN,0.592
4,agua preta,37385,1.0,NaN,2.0,11.0,12.0,26.0,23.0,39.0,30.0,mata meridional pernambucana,NaN,15.0,1.0,NaN


### **Existe uma relação entre o número de nascidos vivos e a população de um determinado grupo de municípios?**

In [39]:
df_born_alive = get_dataframe('df_born_alive')
df_born_alive.head()

,municipio,nascido vivo
0,abreu e lima,1275
1,afogados da ingazeira,575
2,afranio,241
3,agrestina,371
4,agua preta,408


In [40]:
df_merged = get_dataframe('df_merged')
df_merged.head()

,municipio,populacao,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 29 anos,30 a 39 anos,40 a 49 anos,50 a 59 anos,60 a 69 anos,70 a 74 anos,microrregiao,posto de saude,centro de saude/unidade basica,unidade mista,idh
0,abreu e lima,100699,1.0,1.0,8.0,33.0,41.0,68.0,156.0,176.0,114.0,recife,4.0,26.0,NaN,NaN
1,afogados da ingazeira,37545,NaN,1.0,4.0,8.0,12.0,26.0,50.0,43.0,42.0,pajeu,NaN,16.0,NaN,NaN
2,afranio,19978,1.0,NaN,3.0,5.0,8.0,10.0,9.0,17.0,13.0,petrolina,1.0,12.0,NaN,0.588
3,agrestina,25240,NaN,NaN,5.0,8.0,20.0,32.0,15.0,36.0,25.0,brejo pernambucano,16.0,2.0,NaN,0.592
4,agua preta,37385,1.0,NaN,2.0,11.0,12.0,26.0,23.0,39.0,30.0,mata meridional pernambucana,NaN,15.0,1.0,NaN


In [41]:
df_merge = pd.merge(df_merged, df_born_alive, on='municipio')

In [42]:
grouped_df = df_merge.groupby('microrregiao')

correlations = {}
for name, group in grouped_df:
    correlation = group['populacao'].corr(group['nascido vivo'])
    correlations[name] = correlation

for microrregiao, correlation in correlations.items():
    print(f"Correlation for microrregiao '{microrregiao}': {correlation}")

Correlation for microrregiao 'alto capibaribe': 0.9948045380369279
Correlation for microrregiao 'araripina': 0.9801123079225502
Correlation for microrregiao 'brejo pernambucano': 0.9615773220815756
Correlation for microrregiao 'garanhuns': 0.9855522048000062
Correlation for microrregiao 'itamaraca': 0.997824004034205
Correlation for microrregiao 'itaparica': 0.9542679275384821
Correlation for microrregiao 'mata meridional pernambucana': 0.9416119475068051
Correlation for microrregiao 'mata setentrional cha de alegria': nan
Correlation for microrregiao 'mata setentrional pernambucana': 0.9948828353270155
Correlation for microrregiao 'medio capibaribe': 0.9669040105162753
Correlation for microrregiao 'pajeu': 0.9907798760076483
Correlation for microrregiao 'petrolina': 0.9998662234040269
Correlation for microrregiao 'recife': 0.9986671533365433
Correlation for microrregiao 'salgueiro': 0.9918050556689572
Correlation for microrregiao 'sertao do moxoto': 0.9857810234423497
Correlation for 

c:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning:

Degrees of freedom <= 0 for slice

c:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning:

divide by zero encountered in true_divide



### **Quais são os tipos de estabelecimentos de saúde mais comuns nos municípios com os melhores indicadores de saúde básica?**

In [43]:
best_health_indicies = df_health_centers[['municipio', 'taxa de saude disponivel']].sort_values(by='taxa de saude disponivel', ascending=False).municipio[:5].values

In [44]:
df_merged = get_dataframe('df_merged')

In [45]:
places_best_health = df_merged[df_merged['municipio'].isin(best_health_indicies)]

# places_best_health.drop(columns=['municipio'])
places_best_health[['municipio', 'posto de saude', 'centro de saude/unidade basica', 'unidade mista', 'populacao']]

,municipio,posto de saude,centro de saude/unidade basica,unidade mista,populacao
44,caruaru,1.0,68.0,NaN,369342
87,jaboatao dos guararapes,NaN,121.0,NaN,711329
122,paulista,NaN,55.0,1.0,336917
126,petrolina,NaN,58.0,NaN,359366
132,recife,NaN,165.0,NaN,1661019


### **Quais são as principais tendências de mortalidade infantil por cor/raça nos municípios ao longo do tempo?**

In [46]:
df_born_race = get_dataframe('df_born_race')
df_born_race.head()

,causa,Branca,Preta,Amarela,Parda,Indígena,Ignorado
0,..reduziveis atencao a mulher na gestacao,80.0,6.0,NaN,349.0,6.0,29.0
1,..sifilis congenita,1.0,NaN,NaN,9.0,NaN,1.0
2,..feto e recem-nasc afet compl placenta membranas,6.0,NaN,NaN,23.0,NaN,2.0
3,..feto e recem-nascido afet afeccoes maternas,26.0,5.0,NaN,99.0,1.0,12.0
4,..feto e recem-nasc afet por compl matern gravid,16.0,NaN,NaN,77.0,NaN,8.0


In [47]:
races = ['Branca', 'Preta', 'Amarela', 'Parda', 'Indígena']

In [48]:
df_born_race.groupby('causa')[races].sum().idxmax()

Branca      ..reduziveis atencao a mulher na gestacao
Preta       ..reduziveis atencao a mulher na gestacao
Amarela       ..outros riscos acidentais a respiracao
Parda       ..reduziveis atencao a mulher na gestacao
Indígena    ..reduziveis atencao a mulher na gestacao
dtype: object

### **Quais são as principais causas de óbitos evitáveis em diferentes faixas etárias nos municípios com maior concentração populacional?**

In [49]:
df_merged = get_dataframe('df_merged')
df_merged.head()

,municipio,populacao,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 29 anos,30 a 39 anos,40 a 49 anos,50 a 59 anos,60 a 69 anos,70 a 74 anos,microrregiao,posto de saude,centro de saude/unidade basica,unidade mista,idh
0,abreu e lima,100699,1.0,1.0,8.0,33.0,41.0,68.0,156.0,176.0,114.0,recife,4.0,26.0,NaN,NaN
1,afogados da ingazeira,37545,NaN,1.0,4.0,8.0,12.0,26.0,50.0,43.0,42.0,pajeu,NaN,16.0,NaN,NaN
2,afranio,19978,1.0,NaN,3.0,5.0,8.0,10.0,9.0,17.0,13.0,petrolina,1.0,12.0,NaN,0.588
3,agrestina,25240,NaN,NaN,5.0,8.0,20.0,32.0,15.0,36.0,25.0,brejo pernambucano,16.0,2.0,NaN,0.592
4,agua preta,37385,1.0,NaN,2.0,11.0,12.0,26.0,23.0,39.0,30.0,mata meridional pernambucana,NaN,15.0,1.0,NaN


In [50]:
avoidable_deaths_age_group = get_dataframe('avoidable_deaths_age_group')
avoidable_deaths_age_group.head()

,causa,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 29 anos,30 a 39 anos,40 a 49 anos,50 a 59 anos,60 a 69 anos,70 a 74 anos
0,1.1. Reduzíveis pelas ações de imunoprevenção,NaN,NaN,1,NaN,3,10,8.0,11.0,1.0
1,1.2. Reduz ações prom prev contr atenç doenç i...,13.0,7.0,28,173.0,320,548,737.0,1088.0,738.0
2,1.3. Reduz ações prom prev contr atenç doe ñ t...,14.0,20.0,37,230.0,781,2027,4058.0,6054.0,3771.0
3,1.4. Reduz ações prev contr atenção causas matern,NaN,1.0,7,39.0,56,16,NaN,NaN,NaN
4,1.5. Reduz ações prom prev atenç causas externas,27.0,54.0,714,1896.0,1537,1030,833.0,634.0,270.0


In [51]:
age_groups = ['5 a 9 anos', '10 a 14 anos', '15 a 19 anos', '20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos', '60 a 69 anos', '70 a 74 anos']
avoidable_deaths_age_group.groupby('causa')[age_groups].sum().idxmax()

5 a 9 anos            3. Demais causas (não claramente evitáveis)
10 a 14 anos          3. Demais causas (não claramente evitáveis)
15 a 19 anos     1.5. Reduz ações prom prev atenç causas externas
20 a 29 anos     1.5. Reduz ações prom prev atenç causas externas
30 a 39 anos     1.5. Reduz ações prom prev atenç causas externas
40 a 49 anos          3. Demais causas (não claramente evitáveis)
50 a 59 anos          3. Demais causas (não claramente evitáveis)
60 a 69 anos    1.3. Reduz ações prom prev contr atenç doe ñ t...
70 a 74 anos    1.3. Reduz ações prom prev contr atenç doe ñ t...
dtype: object

### **Quais são as diferenças significativas nas taxas de óbito evitáveis entre os municípios de uma determinada região?**

In [52]:
df_born_merged[['municipio', 'taxa de mortes evitaveis']].max()

municipio                   vitoria de santo antao
taxa de mortes evitaveis                       1.0
dtype: object

In [53]:
df_born_merged[['municipio', 'taxa de mortes evitaveis']].min()

municipio                   abreu e lima
taxa de mortes evitaveis        0.200179
dtype: object